In [8]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [10]:
# Lee la poblacion FONASA
ruta_poblaciones = "../data/interim/0.0_poblaciones_ine_y_fonasa_a_utilizar.xlsx"
poblaciones_fonasa = pd.read_excel(ruta_poblaciones, sheet_name="poblacion_FONASA")

# Filtra solamente la poblacion del SSMN y entre 0 y 15 anios (Poblacion que efectivamente
# atendieron). Elimina la columna del 2023, ya que esta mal calculada
poblaciones_fonasa_ssmn = (
    poblaciones_fonasa.query("Estrato == 'SSMN' and `Edad Incidencia` == 'entre_0_y_15'")
    .set_index(["Edad Incidencia", "Estrato"])
    .drop(columns=["2023"])
)
display(poblaciones_fonasa_ssmn)

,,2018,2019,2020,2021,2022
Edad Incidencia,Estrato,,,,,
entre_0_y_15,SSMN,148276,155320,153830,151308,153566


In [35]:
# Lee la poblacion SSMN extrapolada
estratos_interes_0_a_14 = ["recien_nacidos_vivos", "entre_1_y_14"]
estratos_interes_15_a_18 = ["entre_15"]
poblacion_fonasa_extrapolada_ssmn = pd.read_excel(
    ruta_poblaciones, sheet_name="poblaciones_fonasa_extrapoladas"
).query("Estrato == 'SSMN'")

# Extrae solo la poblacion entre 0 y 14 anios
poblacion_fonasa_extrapolada_ssmn_0_a_14 = poblacion_fonasa_extrapolada_ssmn.query(
    "`Edad Incidencia`.isin(@estratos_interes_0_a_14)"
)[ANIOS_POBLACION].sum()


# Extrae solo la poblacion entre 15 y 18 anios
poblacion_fonasa_extrapolada_ssmn_15_a_18 = poblacion_fonasa_extrapolada_ssmn.query(
    "`Edad Incidencia`.isin(@estratos_interes_15_a_18)"
)[ANIOS_POBLACION].sum()

In [22]:
# Lee las consultas de urgencias REM
produccion_urgencias_rem = pd.DataFrame(
    [
        [59, 70, 82, 65, 87, 69, 91, 448, 176, 169],
        [1455, 1292, 2133, 2635, 3096, 1499, 5235, 13613, 12473, 12981],
        [33159, 28148, 25438, 22907, 27428, 13795, 20488, 29809, 28980, 29070],
        [66592, 64954, 63160, 62138, 54425, 21814, 22329, 36763, 37584, 38696],
        [4841, 4781, 0, 0, 1, 0, 0, 0, 0, 0],
        [3764, 3302, 5514, 11914, 8251, 2476, 1484, 933, 1237, 1250],
        [109870, 102547, 96327, 99659, 93288, 39653, 49627, 81566, 80450, 82166],
    ],
    index=["C1", "C2", "C3", "C4", "SC", "C5", "Total"],
    columns=["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]
)
total_urgencias_rem = produccion_urgencias_rem.loc["Total"]

In [ ]:
# Obtiene la proporcion de urgencias y poblacion
proporcion_urgencias_y_poblacion = total_urgencias_rem / poblaciones_fonasa_ssmn

In [28]:
# Selecciona la cantidad de urgenicas por personas.
# De 15 a 18 fue criterio de experto - Dr Tapia
factor_urgencias_0_a_14 = proporcion_urgencias_y_poblacion.max(axis=1).iloc[0]
factor_urgencias_15_a_18 = 1.3

In [ ]:
poblacion_fonasa_extrapolada.query("")

,Edad Incidencia,Estrato,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
147,recien_nacidos_vivos,SSMN,9347.319561,9584.855343,10023.627871,10575.755277,10920.914509,11061.652867,11111.962054,11125.972207,11113.235704,11077.573496,11018.348757,10938.108788,10843.858666,10740.692991,10715.856810,10633.706366,10554.740047,10478.321029,10411.454388
179,entre_1_y_14,SSMN,131096.188691,136025.852182,140735.174172,145201.228953,147907.735844,149571.759962,150765.170295,151676.467085,152358.506822,153038.636083,153675.461233,154278.534651,154602.678653,154546.638039,154509.065355,154881.608069,155334.390751,155569.379231,155392.978665
211,entre_15_y_18,SSMN,36034.751150,36894.465104,37566.952463,38024.192921,38089.149087,38225.429669,38816.403409,39819.403021,40920.473706,41676.385160,42061.027551,42143.177996,42234.243992,42664.100969,42966.592916,42871.069143,42451.401369,41907.552690,41844.507000


In [ ]:
poblaciones_0_y_14_proyectada = poblacion_fonasa_extrapolada

In [ ]:
# Define las poblaciones proyectadas
poblacion_0_a_15_proyectada = 176605
poblacion_15_a_18_proyectada = 41845

# Define cuantas consultas habran por grupo etario
consultas_0_a_15 = poblacion_0_a_15_proyectada * factor_urgencias_0_a_14
consultas_15_a_18 = poblacion_15_a_18_proyectada * factor_urgencias_15_a_18

# Define el total de consultas
total_consultas = consultas_0_a_15 + consultas_15_a_18

print(f"Total consultas urgencia proyectadas 2035: {total_consultas:.2f}")

Total consultas urgencia proyectadas 2035: 173097.94
